In [9]:
from openai import OpenAI
import json
base_url = "http://10.0.0.215:1234/v1"  # Adjust this to your LM Studio server
model = "Meta-Llama-3.1-8B-Instruct-GGUF/Meta-Llama-3.1-8B-Instruct-Q4_K_M.gguf"

client = OpenAI(
    base_url=base_url,
    api_key="asdf",
)


In [10]:
from pydantic import BaseModel

class Planet(BaseModel):
    planet_name: str
    cluster_name: str
    # resources: list[str]
    # special_features: list[str]
    # hazards: list[str]
    # trade_goods: list[str]

class PlanetResponse(BaseModel):
    planets: list[Planet]

In [11]:
# sets
planet_names = {}
cluster_names = {}

In [18]:
def generate_planets(postfix = ""):
  # Generate a Planet name
  question = "Generate an array of planet names. " + postfix
  completion = client.beta.chat.completions.parse(
      model=model,
      messages=[
          {"role": "system", "content": "You are a complete index of all the fantasy alien planets and clusters in the universe. Be creative."},
          {"role": "user", "content": question}
      ],
      response_format=PlanetResponse
    )

  output = json.loads(completion.choices[0].message.content)
  return output


In [13]:
def is_valid_name(name):
  return len(name) > 2 and "(" not in name and ")" not in name

def add_planets(raw_planets):
  for planet in raw_planets['planets']:
    if not is_valid_name(planet['planet_name']):
      print(planet['planet_name'])
    else:
      planet_names[planet['planet_name']] = True
    if not is_valid_name(planet['cluster_name']):
      print(planet['cluster_name'])
    else:
      cluster_names[planet['cluster_name']] = True

In [25]:
raw_planets = generate_planets("Be creative with the cluster names. Make them sound alien and fantastical.")

add_planets(raw_planets)

# list clusters and planets
print(len(planet_names)), print(len(cluster_names))
print(planet_names.keys())
print(cluster_names.keys())


129
78
dict_keys(['Xylonia-IV', "Zha'thik", 'Kraelion-Prime', 'Aetherean-2', 'Nyxoria-Alpha', 'Khyron-Prime', 'Elyria-3', 'Threnody-1', "Mor'Dhul-Prime", 'Sylvaen-IV', 'Xarxos-Prime', 'Aetherean-3', 'Zorvath-Alpha', 'Kaelin-Prime', 'Threnody-3', 'Elyria-1', 'Sylvaen-Prime', 'Ark-IV', 'Ark-VI', 'Necromunda', 'Molech', 'Terra', 'Cadia', 'Perseus Mons', "Molech's sister, Horus' Throne World - Kharon", 'Sarsha', 'Kaelin-IV', "T'Kon", 'Kraelion-III', 'Zorvath-I', "N'Tar", 'Khyron-IV', 'Xeridia-II', "Zhra'gha", 'Rigel IV', "Nal'raxx", 'Deneva', 'Xeridia', 'Rigel V', 'Arkeia-IV', 'Andoria', 'Xylonia', 'Zhilak', 'Rigel II', "Shel'goroth", 'Vrakkor-III', 'Aethereia', 'Thrakos', 'Delta IV', 'Rigel III', 'Elyria', 'Nyxoria', 'Xeridian-IV', 'Rigel X', 'Bajor', 'Lyraxys', 'Zorvath', 'Nebulon-6', 'Deneb IV', 'Tatooine', 'Zorgon Prime', 'Xylophia-IV', 'Rigellian IV', 'Dantooine', 'Nyxoria-III', 'Vega IV', 'Lyraxys-Prime', 'Coruscant', 'Aethereia-Prime', 'Palladium-5', 'Khyron-6', 'Arkona IV', 'Hoth',

In [36]:
file_location = "../pomodoro-space-hauler/data/planet_generation.json"

planet_gen = {}
# read in the existing planet names
with open(file_location, 'r') as f:
    planet_gen = json.load(f)
    planet_gen["cluster_names"] = list(set(planet_gen["cluster_names"]).union(set(cluster_names.keys())))

    planet_gen["planet_names"] = list(set(planet_gen["planet_names"]).union(set(planet_names.keys())))

    # Write out to a json file
    with open(file_location, 'w') as f:
        json.dump(planet_gen, f, indent=2)